In [ ]:
import sys
import os
sys.path.append(os.path.abspath('/home/mshahidul/'))
from gpu_selection import _gpu_selection_
_gpu_selection_()
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer= AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets['train']= raw_datasets["train"].shuffle(seed=42).select(range(1000))

In [ ]:
raw_datasets['train'].features


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
print(samples)
[len(x) for x in samples["input_ids"]]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
batch = data_collator(samples)

{k: v.shape for k, v in batch.items()}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [28]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer",report_to=None)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [30]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer
)

In [31]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=0.08254664611816406, metrics={'train_runtime': 15.0221, 'train_samples_per_second': 199.706, 'train_steps_per_second': 24.963, 'total_flos': 56195154273600.0, 'train_loss': 0.08254664611816406, 'epoch': 3.0})

In [32]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [34]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [35]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.7769607843137255, 'f1': 0.8455008488964346}

In [37]:
def compute_metrics(eval_pred):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [40]:
training_args = TrainingArguments("test-trainer", eval_strategy="epoch")
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,2.091097,0.762255,0.828924
2,No log,2.155973,0.767157,0.841402
3,No log,1.907494,0.769608,0.841216


TrainOutput(global_step=375, training_loss=0.049486974080403645, metrics={'train_runtime': 15.5798, 'train_samples_per_second': 192.557, 'train_steps_per_second': 24.07, 'total_flos': 56195154273600.0, 'train_loss': 0.049486974080403645, 'epoch': 3.0})

In [42]:
training_args = TrainingArguments("test-trainer", eval_strategy="epoch", report_to=None, per_device_train_batch_size=16, 
                                  per_device_eval_batch_size=16,
                                  learning_rate=2e-5, gradient_accumulation_steps=2, lr_scheduler_type="cosine",
                                  )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
